# Spencer Supplemental on Parsing
I was looking through the paring notes and I need to practice my own little language to understand these. So, I typed that up. I figured why not share it with you all and put it in a JN

You’ll notice below that the syntax for a parser is quite advanced. Don’t worry about it right now. We are not going to write one of these **yet** - we’ll do that toward the end of the course.

You’ll note that this does not explain **why** these steps are taken. That is quite intentional. I want you to have this reference of what is done without worrying about why it works

I've also posted my code as a stand alone file that includes tests. The file is named "pythonicBooleanAlgebra.scala"

## High level overview
* We define an object language
* We define a generative grammar
* We implement a class definition in Scala – guided by the generative 
* We define precedence of thing within the language
* We derive a parsing grammar – guided by the generative grammar and the precedence
* We implement a parser – guided by the parsing grammar and the definition of the object language
* We define behavior of operations within the language
* Maybe we bother to write inference rules – not shown in this document
* We implement an interpreter – guided by inference rules of the operator’s behaviors

### Object Language
* The **object language** is the thing I wish to describe. This could be literally any set you can imagine.
* Today I want to look at some simple Boolean algebra, so this could be an acceptable grammar:
    * S &RightArrow; T | ~S | S&&S | S||S | (S)
    * T &RightArrow; 0 | 1
* But I don’t love that one. Instead I’ll use a subset of python: 
    * S &RightArrow;  T | not S | S and S | S or S | (S)
    * T &RightArrow; True | False
* If we prefer we could write it in one line, but I don’t feel like it. Here is an example of that method:
    * S &RightArrow;  True | False | not S | S and S | S or S | (S)

### Generative Grammar
* object language grammar:
    * S &RightArrow;  T | not S | S and S | S or S | (S)
    * T &RightArrow; True | False
* generative grammar: 
	* CondExpr &RightArrow; Bool | Not(CondExpr) | And(CondExpr, CondExpr) | Or(CondExpr, CondExpr)
	* Bool &RightArrow; ConstTrue | ConstFalse
* Observations:
	* the (S) is not shown in the generative grammar, it is not needed… 

### Scala Class
* my generative grammar: 
	* CondExpr &RightArrow; Bool | Not(CondExpr) | And(CondExpr, CondExpr) | Or(CondExpr, CondExpr)
	* Bool &RightArrow; ConstTrue | ConstFalse
* My Class

In [2]:
sealed trait CondExpr
sealed trait Bool extends CondExpr
case object ConstTrue extends Bool
case object ConstFalse extends Bool
case class Not(c1:CondExpr) extends CondExpr
case class Or(c1:CondExpr, c2:CondExpr) extends CondExpr
case class And(c1:CondExpr, c2:CondExpr) extends CondExpr

defined trait CondExpr
defined trait Bool
defined object ConstTrue
defined object ConstFalse
defined class Not
defined class Or
defined class And

### Precedence
* It is entirely your choice how you will define precedence. Today I want to mimic the behavior of python so my precedence will be defined as follows:
* python / general
* our constants, “True” and “False” have the highest precedence
* parentheses heighten the precedence of whatever they encapsulated
* unary operator typically have higher precedence than binary operators
* relative precedence
* {True, False} > {()} > {not} > {or, and}

### Parsing grammars
* object langauge grammar:
    * S &RightArrow;  T | not S | S and S | S or S | (S)
    * T &RightArrow; True | False
* precedence:
    * {True, False} > {()} > {not} > {or, and}
* parsing grammar
	* expr &RightArrow; binary
	* binary &RightArrow; unary | unary and binary | unary or binary
	* unary &RightArrow; atom | not unary
	* atom &RightArrow; bool | (expr)
	* bool &RightArrow; True | False
* Observations:
    * It has the start symbol **expr**
    * It has a chaining structure to it
    * the things with a high precedence are lower in the chaining structure, and would be lower in the parse tree.
    * We write a parsing grammar – guided by the generative grammar and the precedence

### Parser
* generative grammar
	* CondExpr &RightArrow; Bool | Not(CondExpr) | And(CondExpr, CondExpr) | Or(CondExpr, CondExpr)
	* Bool &RightArrow; ConstTrue | ConstFalse
* parsing grammar
	* expr &RightArrow; binary
	* binary &RightArrow; unary and binary | unary or binary | unary
	* unary &RightArrow; atom | not unary
	* atom &RightArrow; bool | (expr)
	* bool &RightArrow; True | False
* parser:
    * works on my machine, but not working on JN right now
    * Something to do with the combinator library


In [2]:
import scala.util.parsing.combinator._
class CondExprParser extends RegexParsers {
    def expr:Parser[CondExpr] = binary
    def binary:Parser[CondExpr] = {
        val opt1 = (unary <~ "and") ~ binary ^^ {
            case e1 ~ e2 => And(e1, e2)
        }
        val opt2 = (unary <~ "or") ~ binary ^^ {
            case e1 ~ e2 => Or(e1,e2)
        }
        val opt3 = unary
        opt1 | opt2 | opt3  // this order counts... GRRRRR
    }
    def unary:Parser[CondExpr] = {
        val opt1 = atom
        val opt2 = ("not" ~> atom) ^^ { Not(_) }
        opt1 | opt2
    }
    def atom:Parser[CondExpr] = {
        val opt1 = bool
        val opt2 = ("(" ~> expr) <~ ")"
        opt1 | opt2
    }
    def bool:Parser[CondExpr] = {
        val opt1 = "True" ^^ { _ => ConstTrue }
        val opt2 = "False" ^^ { _ => ConstFalse }
        opt1 | opt2
    }
}

cmd2.sc:1: object parsing is not a member of package util
import scala.util.parsing.combinator._
                  ^cmd2.sc:2: not found: type RegexParsers
class CondExprParser extends RegexParsers {
                             ^cmd2.sc:3: not found: type Parser
    def expr:Parser[CondExpr] = binary
             ^cmd2.sc:4: not found: type Parser
    def binary:Parser[CondExpr] = {
               ^cmd2.sc:14: not found: type Parser
    def unary:Parser[CondExpr] = {
              ^cmd2.sc:19: not found: type Parser
    def atom:Parser[CondExpr] = {
             ^cmd2.sc:16: value ~> is not a member of String
        val opt2 = ("not" ~> atom) ^^ { Not(_) }
                          ^cmd2.sc:24: not found: type Parser
    def bool:Parser[CondExpr] = {
             ^cmd2.sc:21: value ~> is not a member of String
        val opt2 = ("(" ~> expr) <~ ")"
                        ^cmd2.sc:25: value ^^ is not a member of String
        val opt1 = "True" ^^ { _ => ConstTrue }
                

: 

### Operations Behavior
I learned this with inference rules. I don’t yet know how we’ll cover it this semester. I don’t know how to typeset them well. But here are my inference rules

* OS1: bool = ! bool
	* R1
		* C: ConstFalse = !ConstTrue
	* R2
		* C: ConstTrue = !ConstFalse
* OS2: bool = bool && bool
    * R1
        * P1: bool1 &in; { ConstFalse } OR bool2 &in; { ConstFalse }
        * C: ConstFalse = bool1 && bool2
    * R2
        * P1: bool1 &in; { ConstTrue } AND bool2 &in; { ConstTrue }
        * C: ConstTrue = bool1 && bool2
* OS2: bool = bool || bool
    * R1
        * P1: bool1 &in; { ConstTrue } OR bool2 &in; { ConstTrue}
        * C: ConstTrue = bool1 || bool2
    * R2
        * P1: bool2 not in { ConstTrue } and bool2 not in { ConstTrue }
        * C: ConstFalse = bool1 || bool2
* OS3: bool = expr.eval()
    * R1: 
        * C: bool = bool.eval()
    * R2:
        * P1: bool1 = expr1.eval()
        * P2: bool<sub>out</sub> = !bool1
        * C: bool<sub>out</sub> = !expr1.eval()
    * R3: 
        * P1: bool1 = expr1.eval()
        * P2: bool2 = expr2.eval()
        * P3: bool<sub>out</sub> = bool1 && bool2
        * C: bool<sub>out</sub> = And(expr1, expr2).eval()
    * R4: 
        * P1: bool1 = expr1.eval()
        * P2: bool2 = expr2.eval()
        * P3: bool<sub>out</sub> = bool1 || bool2
        * C: bool<sub>out</sub> = And(expr1, expr2).eval()

### Interpreter
I’ve commented this interpreter… to show the mapping to the operational semantics (op) and the Rules (r) in from above


In [3]:
sealed trait CondExpr {
    // op4
    def eval():Bool = this match {
        // r1
        case b:Bool => b
        // r2
        case Not(c1) => !c1.eval()  // could use (c1 eval but that is discouraged)
        // r3
        case And(c1, c2) => c1.eval() && c2.eval()
        // r4
        case Or(c1, c2) => c1.eval() || c2.eval()
    }
}
sealed trait Bool extends CondExpr {
    // prefix "unary_" allows us to prefix the method
    // so we can say (! b) rather than (b !)
    // op1
    def unary_!():Bool = this match {
        // r1
        case ConstTrue => ConstFalse
        // r2
        case ConstFalse => ConstTrue
    }
    // op2
    def &&(b2:Bool):Bool = (this, b2) match {
        // r1
        case (ConstFalse,_) | (_,ConstFalse) => ConstFalse
        // r2
        case (ConstTrue, ConstTrue) => ConstTrue
    }
    // op3
    def ||(b2:Bool):Bool = (this, b2) match {
        // r1
        case (ConstTrue,_) | (_,ConstTrue) => ConstTrue
        // r2
        case (ConstFalse, ConstFalse) => ConstFalse
    }
}
case object ConstTrue extends Bool
case object ConstFalse extends Bool
case class Not(c1:CondExpr) extends CondExpr
case class Or(c1:CondExpr, c2:CondExpr) extends CondExpr
case class And(c1:CondExpr, c2:CondExpr) extends CondExpr

defined trait CondExpr
defined trait Bool
defined object ConstTrue
defined object ConstFalse
defined class Not
defined class Or
defined class And